In [1]:
import os 
import sys
os.path.dirname(sys.executable)

'c:\\Users\\mhanania\\python\\Lecture de contrats\\AI transformers for contract reading\\.venv\\Scripts'

In [2]:
from datasets import Dataset
import json

label_to_id = {"O": 0, "TITLE": 1, "PERSON": 2, "DATE": 3}


# Charger les données depuis le fichier JSON
with open("generated_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)


dataset = Dataset.from_dict(data)
print(dataset)

c:\Users\mhanania\python\Lecture de contrats\AI transformers for contract reading\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text', 'labels'],
    num_rows: 100
})


In [3]:
from transformers import AutoTokenizer

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

text = "Contrat de partenariat signé par Sophie Leclerc le 5 août 2019."

tokenized_inputs = tokenizer(text, return_offsets_mapping=True, truncation=True, padding=True)

# Récupérer les tokens et leurs offsets
tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
offsets = tokenized_inputs["offset_mapping"]

print("Tokens:", tokens)
print("Offsets:", offsets)

Tokens: ['<s>', '▁Contrat', '▁de', '▁partenariat', '▁signé', '▁par', '▁Sophie', '▁Leclerc', '▁le', '▁5', '▁août', '▁2019.', '</s>']
Offsets: [(0, 0), (0, 7), (8, 10), (11, 22), (23, 28), (29, 32), (33, 39), (40, 47), (48, 50), (51, 52), (53, 57), (58, 63), (0, 0)]


In [4]:
# La phrase initiale, avec ses labels [début, fin, label] label indique quel type de mot est-ce (titre, personne...)
sentence = dataset[0]
print(sentence)

# On passe en token de hugging face
tokenized_inputs = tokenizer(sentence["text"], return_offsets_mapping=True, truncation=True, padding=True)
print(tokenized_inputs)
print(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"]))

tokenized_labels = [0]*len(tokenized_inputs["offset_mapping"])
print(tokenized_labels)

# Le offset associe chaque token à la phrase, ▁Accord est Accord : de 0 à 6 dans la phrase initiale
print(tokenized_inputs["offset_mapping"])
# Les labels sont l'indications du type du mot dans la phrase initiale
print(sentence["labels"])

for i,offset in enumerate(tokenized_inputs["offset_mapping"] ):
    if offset==(0,0) : 
        tokenized_labels[i] = -100

    else:
        for start, end, label in sentence["labels"]:
            if offset[0] >= start and offset[1] <= end:
                tokenized_labels[i] =label
                break

                
print(tokenized_labels)

{'text': 'Accord de collaboration signé par Marc Robert le 15 janvier 2023.', 'labels': [[0, 23, 1], [34, 45, 2], [49, 64, 3]]}
{'input_ids': [5, 18396, 8, 2111, 2917, 37, 3093, 2788, 16, 338, 695, 325, 3009, 9, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 6), (7, 9), (10, 23), (24, 29), (30, 33), (34, 38), (39, 45), (46, 48), (49, 51), (52, 59), (60, 62), (62, 64), (64, 65), (0, 0)]}
['<s>', '▁Accord', '▁de', '▁collaboration', '▁signé', '▁par', '▁Marc', '▁Robert', '▁le', '▁15', '▁janvier', '▁20', '23', '.', '</s>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(0, 0), (0, 6), (7, 9), (10, 23), (24, 29), (30, 33), (34, 38), (39, 45), (46, 48), (49, 51), (52, 59), (60, 62), (62, 64), (64, 65), (0, 0)]
[[0, 23, 1], [34, 45, 2], [49, 64, 3]]
[-100, 1, 1, 1, 0, 0, 2, 2, 0, 3, 3, 3, 3, 0, -100]


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def tokenize_and_align_labels(sentences):
    
    tokenized_inputs = tokenizer(sentences["text"], truncation=True, padding=True, return_offsets_mapping=True)
    labels = []

    for i, sentence_labels in enumerate(sentences["labels"]): # La ieme phrase du batch :  0 , [[0, 23, 1], [34, 45, 2], [49, 64, 3]]
        offsets = tokenized_inputs["offset_mapping"][i] # [(0, 0), (0, 6), (7, 9), (10, 23)... ]
        offset_labels = [0]*len(offsets)

        for j, offset in enumerate(offsets):
            if offset == (0, 0):  # Tokens spéciaux comme <s> ou <pad>
                offset_labels[j] = -100

            else:
                for start, end, label in sentence_labels: # on regarde dans quel interval de sentence_labels est compris le offset
                    if offset[0] >= start and offset[1] <= end:
                        offset_labels[j] = label
                        break

        labels.append(offset_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


# Appliquer la tokenisation et l'alignement. Batched=True sert a executer plusieurs batch de phrases en meme temps
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 100/100 [00:00<00:00, 4905.96 examples/s]


In [6]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

model = AutoModelForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_to_id))

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\mhanania\AppData\Local\Temp\ipykernel_13560\2008178113.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
100%|██████████| 21/21 [01:15<00:00,  3.59s/it]

{'train_runtime': 75.3559, 'train_samples_per_second': 3.981, 'train_steps_per_second': 0.279, 'train_loss': 0.9542125520252046, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.9542125520252046, metrics={'train_runtime': 75.3559, 'train_samples_per_second': 3.981, 'train_steps_per_second': 0.279, 'total_flos': 2602807725600.0, 'train_loss': 0.9542125520252046, 'epoch': 3.0})

In [7]:
model.save_pretrained("./custom_camembert_ner")
tokenizer.save_pretrained("./custom_camembert_ner")


('./custom_camembert_ner\\tokenizer_config.json',
 './custom_camembert_ner\\special_tokens_map.json',
 './custom_camembert_ner\\tokenizer.json')

## Charger le modèle 

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Charger le modèle et le tokenizer
model = AutoModelForTokenClassification.from_pretrained("./custom_camembert_ner")
tokenizer = AutoTokenizer.from_pretrained("./custom_camembert_ner")


# Texte à analyser
text = "Avenant au CTR24585: SOLUTEC 15/03/2023"

# Tokeniser le texte
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)


In [9]:
# Obtenir les logits (scores bruts)
outputs = model(**inputs)
logits = outputs.logits

# Obtenir les prédictions
predictions = torch.argmax(logits, dim=2)  # Dim=2 correspond à la dimension des labels

# Afficher les résultats
print(predictions)


tensor([[0, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0]])


In [10]:
# Associer les tokens et les prédictions
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
labels = predictions[0].tolist()

# Remettre les étiquettes au format lisible
label_map = {1: "TITLE", 2: "PERSON", 3: "DATE"}
results = []

for token, label_idx in zip(tokens, labels):
    if label_idx in label_map.keys(): # Ignorer les tokens spéciaux
        results.append((token, label_map[label_idx]))

# Afficher les résultats
print("Predictions:")
for token, label in results:
    print(f"{token}: {label}")


Predictions:
▁A: TITLE
venant: TITLE
▁au: TITLE
▁C: TITLE
TR: DATE
2: DATE
45: DATE
85: DATE
:: DATE
▁SOL: DATE
UTE: DATE
C: DATE
▁15: DATE
/03/: DATE
20: DATE
23: DATE


# Test sur une dizaine de phrases

In [11]:
test_sentences = {
    "text":["Contrat d'achat signé par Camille Lefèvre le 15 mars 2022.",
            "Accord de coopération approuvé par Mathieu Girard le 1er avril 2021.",
            "Convention de stage validée par Chloé Morel le 12 mai 2020.",
            "Contrat de licence signé par Victor Dumas le 18 juin 2023.",
            "Accord de sous-traitance confirmé par Emma Laurent le 3 septembre 2022.",
            "Accord technique validé par Thomas Perrin le 20 février 2021.",
            "Contrat d'exploitation signé par Léa Vidal le 7 octobre 2020.",
            "Convention de partenariat officialisée par Hugo Charpentier le 14 août 2019.",
            "Contrat de développement signé par Zoé Lambert le 5 novembre 2021.",
            "Contrat de maintenance validé par Arthur Simon le 23 janvier 2022."
    ],
    "labels":[
        [(0, 15, 1), (25, 39, 2), (43, 57, 3)],
        [(0, 21, 1), (35, 50, 2), (54, 69, 3)],
        [(0, 19, 1), (31, 43, 2), (47, 60, 3)],
        [(0, 19, 1), (30, 42, 2), (46, 60, 3)],
        [(0, 26, 1), (40, 53, 2), (57, 74, 3)],
        [(0, 17, 1), (29, 43, 2), (47, 62, 3)],
        [(0, 23, 1), (34, 43, 2), (47, 62, 3)],
        [(0, 25, 1), (39, 56, 2), (60, 74, 3)],
        [(0, 25, 1), (36, 47, 2), (51, 68, 3)],
        [(0, 24, 1), (36, 47, 2), (51, 66, 3)]
    ]
}
test_data = tokenize_and_align_labels(test_sentences)

outputs = model(input_ids=torch.tensor(test_data["input_ids"]))
logits = outputs.logits
predictions = torch.argmax(logits, dim=2).tolist()


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [12]:
for i in range (len(predictions)):
    print(f"pred :{predictions[i]} - reel : {test_data['labels'][i]}")

pred :[0, 1, 1, 0, 1, 0, 0, 2, 2, 3, 2, 0, 3, 3, 3, 0, 0] - reel : [-100, 1, 1, 1, 1, 0, 0, 2, 2, 2, 0, 0, 3, 3, 3, 0, -100]
pred :[0, 1, 1, 1, 0, 0, 2, 2, 0, 3, 3, 3, 3, 3, 0, 0, 0] - reel : [-100, 1, 1, 1, 0, 0, 2, 2, 0, 0, 3, 3, 3, 3, 3, -100, -100]
pred :[0, 1, 1, 1, 0, 0, 2, 2, 2, 0, 3, 3, 3, 0, 0, 0, 0] - reel : [-100, 1, 1, 1, 0, 0, 2, 2, 2, 0, 3, 3, 3, 3, -100, -100, -100]
pred :[0, 1, 1, 1, 0, 0, 2, 2, 2, 0, 3, 3, 3, 3, 0, 0, 0] - reel : [-100, 1, 1, 1, 0, 0, 0, 2, 2, 0, 0, 3, 3, 3, 3, -100, -100]
pred :[0, 1, 1, 1, 1, 1, 0, 0, 2, 2, 0, 3, 3, 3, 0, 0, 0] - reel : [-100, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 0, 0, 3, 3, -100, -100]
pred :[0, 1, 1, 0, 0, 2, 2, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0] - reel : [-100, 1, 1, 0, 0, 0, 2, 0, 0, 3, 3, 3, 3, -100, -100, -100, -100]
pred :[0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 3, 3, 3, 0, 0, 0, 0] - reel : [-100, 1, 1, 1, 1, 0, 0, 0, 2, 0, 0, 3, 3, 3, -100, -100, -100]
pred :[0, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 0, 3, 3, 3, 0, 0] - reel : [-100, 1, 1, 1, 0, 0, 2, 2, 

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_true, y_pred = [], []

for i, label_seq in enumerate(test_data["labels"]):
    for j, label in enumerate(label_seq):
        if label != -100:  # Ignorer les tokens spéciaux
            y_true.append(label)
            y_pred.append(predictions[i][j])

precision = precision_score(y_true, y_pred, average="weighted")
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

print(f"Précision: {precision:.2f}, Accuracy: {accuracy:.2f}, Rappel: {recall:.2f}, F1-Score: {f1:.2f}")

Précision: 0.77, Accuracy: 0.77, Rappel: 0.77, F1-Score: 0.77


# Test sur un vrai document

In [14]:
from Pdf2Text import pdf_to_text
path="modeles_de_contrats-convention_d_accord-cap.pdf"
text = pdf_to_text(path)

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Obtenir les logits (scores bruts)
outputs = model(**inputs)
logits = outputs.logits

# Obtenir les prédictions
predictions = torch.argmax(logits, dim=2)  # Dim=2 correspond à la dimension des labels

# Associer les tokens et les prédictions
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
labels = predictions[0].tolist()

# Remettre les étiquettes au format lisible
label_map = {1: "TITLE", 2: "PERSON", 3: "DATE"}
results = []

for token, label_idx in zip(tokens, labels):
    if label_idx in label_map.keys(): # Ignorer les tokens spéciaux
        results.append((token, label_map[label_idx]))

# Afficher les résultats
print("Predictions:")
for token, label in results:
    print(f"{token}: {label}")


Predictions:
▁Convention: TITLE
▁d: TITLE
’: DATE
accord: TITLE
▁(: DATE
indi: DATE
▁Jean: PERSON
▁B: DATE
aliste: DATE
▁que: DATE
r: DATE
▁Pierre: PERSON
▁D: TITLE
’: DATE
ette: DATE
indiquer: DATE
▁convention: TITLE
▁d: TITLE
accord: TITLE
s: TITLE
▁but: DATE
▁définitivement: TITLE
▁litige: TITLE
▁de: TITLE
▁...: TITLE
▁nature: DATE
▁litige: TITLE
▁voulez: DATE
:: DATE
▁contra: TITLE
▁t: TITLE
▁de: TITLE
▁vente: TITLE
▁du: TITLE
▁...: TITLE
▁ou: TITLE
▁contrat: TITLE
▁d: TITLE
’: DATE
entreprise: TITLE
▁pour: DATE
▁...: DATE
▁conviennent: TITLE
▁cet: DATE
▁accord: TITLE
▁1.: DATE
▁Prestation: TITLE
s: TITLE
▁de: TITLE
▁Jean: PERSON
▁B: DATE
aliste: DATE
indiquer: DATE
▁votre: PERSON
▁prestations: TITLE
▁prestation: TITLE
oubliez: DATE
▁paragraphe: DATE
▁prestation: TITLE
▁fournir: TITLE
▁prestations: TITLE
▁prestations: TITLE
▁accessoires: TITLE
▁manière: DATE
▁d: DATE
▁ont: DATE
▁la: DATE
▁prestation: TITLE
▁être: DATE
▁paiement: TITLE
▁de: TITLE
▁de: TITLE
▁prestation: TITLE
▁droit